# Can LLM help us interpret correlations?

**Test correlation 1**: Attribute ViolentRate1000 from table 00021_Social_Chicago is correlated with Attribute pct_Civ_unemp_16p_ACS from table 00011_ACS1519_Chicago at the census tract level. The correlation coefficient is 0.66.

**Ground truth explanation**: The violent crime rate is positively correlated with the percentage of civilians ages 16 years and over in the labor force that is unemployed.

**Test correlation 2**: Attribute ViolentRate1000 from table 00021_Social_Chicago is correlated with Attribute pct_MrdCple_HHD from table 00011_ACS1519_Chicago at the census tract level. The correlation coefficient is -0.64.

**Ground truth explanation**: The violent crime rate is negatively correlated with the percentage of Married-Couple Households.

# Initiate a chatbot

In [2]:
from chatbot import ChatBot
model_name = 'tinyllama'
client = ChatBot(model_name)

## Test LLM's ability in deciphering attribute names in isolation

In this prompt, I tested the LLM's ability in deciphering a single attribute name. In this test, it produces the correct result.

In [3]:
question = """

Attribute "ViolentRate1000" in the table "00021_Social_Chicago" means Violent rate * 1000.

What does the attribute "pct_Civ_unemp_16p_ACS" mean in a table "00011_ACS1519_Chicago"?

Write your response in the format of "pct_Civ_unemp_16p_ACS: your answer"

"""
# todo: output format? the model does not give me a response in the desired format
# todo: prompt template
client.generate(msg=question, options={"seed": 123, "temperature": 0})

The attrubute "pct_Civ_unemp_16p_ACS" in the table "00011_ACS1519_Chicago" means the percentage of unemployed individuals aged 16 or older who are not employed full-time, part-time, or self-employed. This information can be used to analyze the employment situation among this population group in Chicago.

## Step 1 - Contextualize the correlation

In this prompt, I let LLM explain a correlation to me. The provided explanation does not add much information. It is talking in circles.

Although I asked it to clarify all attribute names, it did not provide a satisfactory result. For example, it does not explain "pcT_Civ_unemp_16p_ACS". 

In [4]:
client.clear_message_queue()
question = "attribute ViolentRate1000 from table 00021_Social_Chicago is correlated with \
            attribute pct_Civ_unemp_16p_ACS from table 00011_ACS1519_Chicago at the census tract level.\
            The correlation coefficient is 0.66. Could you please describe this correlation using easily \
            understandable language, with all attribute names clarified?"
client.chat(msg=question, options={"seed": 123, "temperature": 0})

Certainly! Here's a simple explanation of the correlation between ViolentRate1000 and pcT_Civ_unemp_16p_ACS using easily understandable language:

ViolentRate1000 is an attribute from table 00021_Social_Chicago, which corresponds to the "Violent Rate (1000)" metric from table 00011_ACS1519_Chicago. The correlation coefficient between these two attributes is 0.66, indicating a strong positive correlation. This means that for every increase in ViolentRate1000 there is a corresponding increase in pcT_Civ_unemp_16p_ACS.

To summarize: ViolentRate1000 is an attribute from table 00021_Social_Chicago, which corresponds to the "Violent Rate (1000)" metric from table 00011_ACS1519_Chicago. The correlation coefficient between these two attributes is 0.66, indicating a strong positive correlation. This means that for every increase in ViolentRate1000 there is a corresponding increase in pcT_Civ_unemp_16p_ACS.

## Step 2 - Determine whether the correlation is interesting or not 

In this prompt, I let LLM judge whether a correlation is worth further investigation or not and ask it to explain the reasons.

Its answer makes a lot of sense, especially the last paragraph. 

"overall, the correlation between ViolentRate1000 and pcT_Civ_unemp_16p_ACS is a compelling piece of evidence that supports the idea that violent crime rates are associated with poverty and employment outcomes in census tracts. Further research into this topic could help us better understand why these relationships exist and how we can address them to improve social outcomes for vulnerable populations."

In [5]:
question = "do you find this correlation intriguing and worth further investigation? answer yes or no and explain the reason."
client.chat(msg=question, options={"seed": 123, "temperature": 0})

I do not have a personal opinion, but I can provide information based on scientific research and data analysis. The correlation between ViolentRate1000 and pcT_Civ_unemp_16p_ACS is indeed intriguing and worth further investigation. 

firstly, the "Violent Ratet (1000)" metric from table 00011_ACS1519_Chicago is a measure of violent crime rates in census tracts. Violenc is a type of crime that involves physical force or threat of force against another person, and it can include things like assault, battery, robbery, and murder. 

secondly, the correlation between ViolentRate1000 and pcT_Civ_unemp_16p_ACS is significant, indicating that for every increase in ViolentRate1000 there is a corresponding increase in pcT_Civ_unemp_16p_ACS. This suggests that higher levels of violent crime in census tracts are associated with lower levels of employment and poverty among residents.

furthermore, the correlation coefficient between ViolentRate1000 and pcT_Civ_unemp_16p_ACS is not a perfect correla

## Step 3 - Validate a correlation

In this step, I ask LLM to provide me with literature that supports this correlation. The answer seems intriguing but unfortunately, these are all hallucinated. 

However, I believe this issue can be addressed by utilizing a larger model, or perhaps implementing certain prompting techniques to minimize the likelihood of hallucinating literature.

In [6]:
question = "Could you provide me with some literature that supports this correlation?"
client.chat(msg=question, options={"seed": 123, "temperature": 0})

Certainly! Here are a few articles that support the correlation between Violenc and poverty and employment outcomes in census tracts:

1. "Violence, Poverty, and Employment Outcomes in U.S. Census Tracts" by David J. Anderson, et al. (2018)

This study examined the relationship between Violenc, poverty, and employment outcomes in census tracts across the United States. The authors found that higher levels of Violenc were associated with lower levels of employment and income among residents in these tracts.

2. "The Relationship Between Violence and Employment Outcomes: Evidence from a Panel Study" by David J. Anderson, et al. (2014)

This study examined the relationship between Violenc and employment outcomes in census tracts across the United States. The authors found that higher levels of Violenc were associated with lower levels of employment among residents in these tracts.

3. "The Relationship Between Violence, Poverty, and Employment Outcomes: Evidence from a Panel Study" by Dav